## Import

In [1]:
import pandas as pd
import random
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

## Data Load

In [ ]:
# make [] 

train_col_year = []
train_col_month = []
train_col_day = []

test_col_year = []
test_col_month = []
test_col_day = []


# split : year, month, day

for i in range(len(train_time_df)):
    time_df_i = train_time_df[i]    
    year = time_df_i[:4]
    train_col_year.append(year)

    month = time_df_i[5:7]
    train_col_month.append(month)

    day = time_df_i[8:10]
    train_col_day.append(day)

for i in range(len(test_time_df)):
    time_df_i = test_time_df[i]    
    year = time_df_i[:4]
    test_col_year.append(year)

    month = time_df_i[5:7]
    test_col_month.append(month)

    day = time_df_i[8:10]
    test_col_day.append(day)


# 
train_df['year'] = train_col_year
train_df['month'] = train_col_month
train_df['day'] = train_col_day

test_df['year'] = test_col_year
test_df['month'] = test_col_month
test_df['day'] = test_col_day

# astype
train_df = train_df.astype({'year':'int'})
train_df = train_df.astype({'month':'int'})
train_df = train_df.astype({'day':'int'})

test_df = test_df.astype({'year':'int'})
test_df = test_df.astype({'month':'int'})
test_df = test_df.astype({'day':'int'})

In [3]:
import pandas as pd
import numpy as np


train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

train_time_df = train_df['TIMESTAMP']
test_time_df = test_df['TIMESTAMP']

def split_TIMESTEPS(train_df, test_df, train_time_df, test_time_df):

    # make [] 

    train_col_year = []
    train_col_month = []
    train_col_day = []

    test_col_year = []
    test_col_month = []
    test_col_day = []


    # split : year, month, day

    for i in range(len(train_time_df)):
        time_df_i = train_time_df[i]    
        year = time_df_i[:4]
        train_col_year.append(year)

        month = time_df_i[5:7]
        train_col_month.append(month)

        day = time_df_i[8:10]
        train_col_day.append(day)
    
    for i in range(len(test_time_df)):
        time_df_i = test_time_df[i]    
        year = time_df_i[:4]
        test_col_year.append(year)

        month = time_df_i[5:7]
        test_col_month.append(month)

        day = time_df_i[8:10]
        test_col_day.append(day)


    # 
    train_df['year'] = train_col_year
    train_df['month'] = train_col_month
    train_df['day'] = train_col_day

    test_df['year'] = test_col_year
    test_df['month'] = test_col_month
    test_df['day'] = test_col_day

split_TIMESTEPS(train_df, test_df, train_time_df, test_time_df)

In [4]:
# astype

train_df = train_df.astype({'year':'int'})
train_df = train_df.astype({'month':'int'})
train_df = train_df.astype({'day':'int'})

test_df = test_df.astype({'year':'int'})
test_df = test_df.astype({'month':'int'})
test_df = test_df.astype({'day':'int'})

In [5]:
train_df.describe()

,Y_Class,Y_Quality,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8,...,X_2866,X_2867,X_2868,X_2869,X_2870,X_2871,X_2872,X_2873,X_2874,X_2875
count,598.000000,598.000000,349.000000,349.000000,349.0,349.0,349.000000,349.0,349.000000,349.000000,...,100.000000,100.00000,100.000000,100.000000,100.00000,99.0,0.0,0.0,0.0,0.0
mean,1.025084,0.530896,2.409742,95.123209,0.0,45.0,10.392550,0.0,48.802292,10.048711,...,50.807300,53.60770,49.606200,51.659800,66.64970,1.0,NaN,NaN,NaN,NaN
std,0.565069,0.007401,5.895256,4.107640,0.0,0.0,0.489019,0.0,4.373824,0.215571,...,7.011828,8.13899,7.158917,8.913065,4.52781,0.0,NaN,NaN,NaN,NaN
min,0.000000,0.500856,1.000000,87.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,...,32.120000,31.70000,32.560000,30.490000,61.67000,1.0,NaN,NaN,NaN,NaN
25%,1.000000,0.527535,2.000000,93.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,...,49.485000,52.20000,42.160000,49.915000,63.64500,1.0,NaN,NaN,NaN,NaN
50%,1.000000,0.530436,2.000000,95.000000,0.0,45.0,10.000000,0.0,45.000000,10.000000,...,53.425000,55.92500,51.460000,56.175000,65.14000,1.0,NaN,NaN,NaN,NaN
75%,1.000000,0.533433,2.000000,98.000000,0.0,45.0,11.000000,0.0,51.000000,10.000000,...,55.287500,58.97500,55.030000,57.175000,67.11500,1.0,NaN,NaN,NaN,NaN
max,2.000000,0.578841,103.000000,102.000000,0.0,45.0,11.000000,0.0,62.000000,11.000000,...,60.240000,68.66000,60.410000,59.930000,79.75000,1.0,NaN,NaN,NaN,NaN


In [5]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'Y_Class', 'Y_Quality', 'TIMESTAMP'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])

## Data Pre-processing

In [6]:
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)

In [7]:
# qualitative to quantitative
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])
    
    for label in np.unique(test_x[i]): 
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i]) 
print('Done.')

Done.


## Split train / valid

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train_x, train_y, random_state=42, test_size=0.2)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)

(478, 2880)
(478,)
(120, 2880)
(120,)


## Classification Model Fit

### Train / Valid

In [9]:
### 나중에 optuna 적용해보기 ###


from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

def model_fit_pred(x_train, y_train, x_val, y_val):
    
    ### RandomForestClassifier ###
    RF = RandomForestClassifier(random_state=42).fit(x_train, y_train)
    print('RandomForestClassifier >>>>> Done', '\n\n')
    rf_pred = RF.predict(x_val)
    print(classification_report(y_val, rf_pred))

    ### XGBClassifier ###
    XGB = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.01, objective='multi:softmax').fit(x_train, y_train)
    print('\n\n', 'XGBClassifier >>>>> Done', '\n\n')
    xgb_pred = XGB.predict(x_val)
    print(classification_report(y_val, xgb_pred))   

    submit = pd.read_csv('./sample_submission.csv')
    submit['Y_Class'] = xgb_pred
    submit.to_csv('./xgb_submission.csv', index=False)



In [59]:
model_fit_pred(x_train, y_train, x_val, y_val)

RandomForestClassifier >>>>> Done 


              precision    recall  f1-score   support

           0       0.78      0.30      0.44        23
           1       0.72      0.96      0.82        76
           2       0.78      0.33      0.47        21

    accuracy                           0.73       120
   macro avg       0.76      0.53      0.57       120
weighted avg       0.74      0.72      0.68       120

[00:00:21] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 XGBClassifier >>>>> Done 


              precision    recall  f1-score   support

           0       0.86      0.26      0.40        23
           1       0.70      0.96      0.81        76
           2       0.78      0.33  

(array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
        1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 0, 1, 1, 1]),
 array([1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,
        2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2,
        1, 1, 1, 0, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1]))

#### RF

In [11]:
### RF ###

RF = RandomForestClassifier(random_state=42).fit(x_train, y_train)
print('Done.')

from sklearn.metrics import classification_report

pred = RF.predict(x_val)
print(classification_report(y_val, pred))

Done.
              precision    recall  f1-score   support

           0       0.78      0.30      0.44        23
           1       0.72      0.96      0.82        76
           2       0.78      0.33      0.47        21

    accuracy                           0.73       120
   macro avg       0.76      0.53      0.57       120
weighted avg       0.74      0.72      0.68       120



##### optuna

In [46]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
import optuna


n_trials = 5000

def objective_rfc(trial):

    rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
    rf_n_estimators = trial.suggest_int("n_estimators", 1, 10000, log=True)
    classifier_obj = RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=rf_n_estimators
        )
    
    rf_n_jobs = trial.suggest_int("n_jobs", 1, 15, log=True)
    rc_cv = trial.suggest_int("cv", 2, 15, log=True)
    score = sklearn.model_selection.cross_val_score(classifier_obj, x_train, y_train, n_jobs=-rf_n_jobs, cv=rc_cv)
    accuracy = score.mean()
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_rfc, n_trials=n_trials, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-02-04 12:00:59,797] A new study created in memory with name: no-name-cf1c0056-8031-4c02-a856-64257bea352a
[I 2023-02-04 12:01:01,151] Trial 0 finished with value: 0.6862159329140461 and parameters: {'rf_max_depth': 30, 'n_estimators': 3, 'n_jobs': 2, 'cv': 3}. Best is trial 0 with value: 0.6862159329140461.
[I 2023-02-04 12:01:40,976] Trial 1 finished with value: 0.7468881856540084 and parameters: {'rf_max_depth': 26, 'n_estimators': 9919, 'n_jobs': 1, 'cv': 6}. Best is trial 1 with value: 0.7468881856540084.
[I 2023-02-04 12:01:41,990] Trial 2 finished with value: 0.7322175732217573 and parameters: {'rf_max_depth': 26, 'n_estimators': 342, 'n_jobs': 13, 'cv': 2}. Best is trial 1 with value: 0.7468881856540084.
[I 2023-02-04 12:01:42,813] Trial 3 finished with value: 0.698936170212766 and parameters: {'rf_max_depth': 29, 'n_estimators': 6, 'n_jobs': 2, 'cv': 10}. Best is trial 1 with value: 0.7468881856540084.
[I 2023-02-04 12:01:43,430] Trial 4 finished with value: 0.751061320

Number of finished trials:  35
Best trial:
  Value: 0.7700645475581537
  Params: 
    rf_max_depth: 19
    n_estimators: 972
    n_jobs: 7
    cv: 7


#### XGB

In [ ]:
# xgb (1)

import xgboost as xgb

dtrain = xgb.DMatrix(x_train, y_train)
dval = xgb.DMatrix(x_val, y_val)

# specify parameters via map
param = {"max_depth": 100, "eta": 1, "objective": "multi:softmax"}

# specify validations set to watch performance
watchlist = [(dval, "eval"), (dtrain, "train")]

# number of boosting rounds
num_round = 2
bst = xgb.train(param, dtrain, num_boost_round=num_round, evals=watchlist)
print('XGBClassifier >>>>> Done', '\n\n')

# run prediction
preds = bst.predict(dval)
labels = dval.get_label()


In [11]:
# xgb (2)

XGB = XGBClassifier(n_estimators=1000, max_depth=3, learning_rate=0.01, objective='multi:softmax').fit(x_train, y_train)
print('\n\n', 'XGBClassifier >>>>> Done', '\n\n')
xgb_pred = XGB.predict(x_val)
print(classification_report(y_val, xgb_pred))  

# submit = pd.read_csv('./sample_submission.csv')
# submit['Y_Class'] = xgb_pred
# submit.to_csv('./xgb_submission.csv', index=False)

[09:39:57] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 XGBClassifier >>>>> Done 


              precision    recall  f1-score   support

           0       0.83      0.22      0.34        23
           1       0.71      0.95      0.81        76
           2       0.67      0.38      0.48        21

    accuracy                           0.71       120
   macro avg       0.74      0.52      0.55       120
weighted avg       0.72      0.71      0.66       120



##### optuna (1)

In [47]:
import pandas as pd
import xgboost as xgb
import sklearn.metrics
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets

n_trials = 5000 # <<<<<

# 아래 함수에서 파라미터 조정
def objective_XGB(trial):
    
    dtrain = xgb.DMatrix(x_train, y_train)
    dvalid = xgb.DMatrix(x_val, y_val)

    param = {
        "verbosity": 0,
        
        "objective": "multi:softmax",   # 👈  binary:logistic >> multi:softmax 으로 변경
        "num_class": '3',  # 👈 "objective": "multi:softmax" 일 시 num_class 파라미터 지정해야 해서 추가함
        
        #  https://romg2.github.io/mlguide/03_머신러닝-완벽가이드-04.-분류-XGBoost/  👈 참고사이트


        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 2, 32, log=True)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.accuracy_score(y_val, pred_labels)
    return accuracy

# 아래 함수에서 실행
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_XGB, n_trials=n_trials, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-02-04 12:12:34,505] A new study created in memory with name: no-name-e8689ffb-83bf-44cc-93cb-d5e0e7018e82
[I 2023-02-04 12:12:34,733] Trial 0 finished with value: 0.7083333333333334 and parameters: {'booster': 'gblinear', 'lambda': 0.0008460832664166152, 'alpha': 1.474384133969996e-06, 'subsample': 0.5745203087098282, 'colsample_bytree': 0.720347682454879}. Best is trial 0 with value: 0.7083333333333334.
[I 2023-02-04 12:12:35,197] Trial 1 finished with value: 0.6833333333333333 and parameters: {'booster': 'dart', 'lambda': 1.2355455919076258e-05, 'alpha': 1.4861345788651963e-05, 'subsample': 0.3147168419397044, 'colsample_bytree': 0.8838303390301048, 'max_depth': 6, 'min_child_weight': 8, 'eta': 0.5333675779808008, 'gamma': 0.6157273989877262, 'grow_policy': 'depthwise', 'sample_type': 'uniform', 'normalize_type': 'forest', 'rate_drop': 4.388250819172249e-08, 'skip_drop': 0.007444371331961654}. Best is trial 0 with value: 0.7083333333333334.
[I 2023-02-04 12:12:35,522] Trial 2

Number of finished trials:  1022
Best trial:
  Value: 0.7916666666666666
  Params: 
    booster: dart
    lambda: 0.2553839284082131
    alpha: 2.963365197931572e-05
    subsample: 0.5476984339603794
    colsample_bytree: 0.5739754370625748
    max_depth: 11
    min_child_weight: 3
    eta: 6.065247920269048e-06
    gamma: 0.7336338498338038
    grow_policy: lossguide
    sample_type: weighted
    normalize_type: forest
    rate_drop: 3.9072533559090704e-07
    skip_drop: 0.00014872786300187534


Number of finished trials:  1022
Best trial:
  Value: 0.7916666666666666
  Params: 
    booster: dart
    lambda: 0.2553839284082131
    alpha: 2.963365197931572e-05
    subsample: 0.5476984339603794
    colsample_bytree: 0.5739754370625748
    max_depth: 11
    min_child_weight: 3
    eta: 6.065247920269048e-06
    gamma: 0.7336338498338038
    grow_policy: lossguide
    sample_type: weighted
    normalize_type: forest
    rate_drop: 3.9072533559090704e-07
    skip_drop: 0.00014872786300187534

In [57]:
#Number of finished trials:  1022
#Best trial:
#  Value: 0.7916666666666666

dtrain = xgb.DMatrix(train_x, train_y)
dvalid = xgb.DMatrix(test_x)

param={'Pbooster': 'dart',
'lambda': 0.2553839284082131,
'alpha': 2.963365197931572e-05,
'subsample': 0.5476984339603794,
'colsample_bytree': 0.5739754370625748,
'max_depth': 11,
'min_child_weight': 3,
'eta': 6.065247920269048e-06,
'gamma': 0.7336338498338038,
'grow_policy': 'lossguide',
'sample_type': 'weighted',
'normalize_type': 'forest',
'rate_drop': 3.9072533559090704e-07,
'skip_drop': 0.00014872786300187534}

bst = xgb.train(param, dtrain)
preds = bst.predict(dvalid)
xgb_optuna_labels = np.rint(preds)
#accuracy = sklearn.metrics.accuracy_score(y_val, xgb_optuna_labels)

In [58]:
submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = xgb_optuna_labels
submit.to_csv('./xgb_optuna_submission.csv', index=False)

##### optuna (2)

In [44]:
import pandas as pd
import xgboost as xgb
import sklearn.metrics
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets

n_trials = 100 # <<<<<

# 아래 함수에서 파라미터 조정
def objective_XGB(trial):
    
    dtrain = xgb.DMatrix(x_train, y_train)
    dvalid = xgb.DMatrix(x_val, y_val)

    param = {
        "verbosity": 0,
        
        "objective": "multi:softmax",   # 👈  binary:logistic >> multi:softmax 으로 변경
        "num_class": '3',  # 👈 "objective": "multi:softmax" 일 시 num_class 파라미터 지정해야 해서 추가함
        
        #  https://romg2.github.io/mlguide/03_머신러닝-완벽가이드-04.-분류-XGBoost/  👈 참고사이트


        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
    
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 2, 32, log=True)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    # bst = xgb.train(param, dtrain)
    # preds = bst.predict(dvalid)
    # pred_labels = np.rint(preds)


    XGB_class = xgb()

    rf_n_jobs = trial.suggest_int("n_jobs", 1, 15, log=True)
    rc_cv = trial.suggest_int("cv", 2, 15, log=True)
    score = sklearn.model_selection.cross_val_score(XGB_class, x_train, y_train, n_jobs=-rf_n_jobs, cv=rc_cv)
    accuracy = score.mean()
    return accuracy

# 아래 함수에서 실행
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_XGB, n_trials=n_trials, timeout=600)

    print("Number of finished trials: ", len(study.trials))
    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))





n_trials = 10

def objective_rfc(trial):

    rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
    rf_n_estimators = trial.suggest_int("n_estimators", 1, 100, log=True)
    classifier_obj = RandomForestClassifier(
            max_depth=rf_max_depth, n_estimators=rf_n_estimators
        )
    
    rf_n_jobs = trial.suggest_int("n_jobs", 1, 15, log=True)
    rc_cv = trial.suggest_int("cv", 2, 15, log=True)
    score = sklearn.model_selection.cross_val_score(classifier_obj, x_train, y_train, n_jobs=-rf_n_jobs, cv=rc_cv)
    accuracy = score.mean()
    return accuracy




[I 2023-02-04 10:23:38,962] A new study created in memory with name: no-name-fa6eda7c-bc78-4647-a83c-04097cb019d6
[W 2023-02-04 10:23:39,042] Trial 0 failed because of the following error: TypeError("'module' object is not callable")
Traceback (most recent call last):
  File "/Users/jeong-geun-o/miniconda3/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/w4/fbwcp4j13tldxpsj0dtj__7r0000gn/T/ipykernel_24475/1344757676.py", line 61, in objective_XGB
    XGB_class = xgb()
TypeError: 'module' object is not callable


TypeError: 'module' object is not callable

#### Train / Test

In [62]:
XGB = XGBClassifier(n_estimators=500, max_depth=2, learning_rate=0.01, objective='multi:softmax').fit(train_x, train_y)
print('\n\n', 'XGBClassifier >>>>> Done', '\n\n')
xgb_pred = XGB.predict(test_x)

[00:10:15] WARNING: /var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4f_b8pp6bp/croot/xgboost-split_1675119661934/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


 XGBClassifier >>>>> Done 




In [63]:
submit = pd.read_csv('./sample_submission.csv')
submit['Y_Class'] = xgb_pred
submit.to_csv('./xgb_submission.csv', index=False)

## Inference

In [ ]:
preds = RF.predict(test_x)
print('Done.')

Done.


## Submit

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

submit['Y_Class'] = preds

submit.to_csv('./baseline_submission.csv', index=False)